# Compile Dataframe from raw subject files

In [1]:
import pandas as pd; import os; from analysis_helpers import *; import warnings
warnings.filterwarnings('ignore')

In [3]:
sub_list = []

data_dir = '/Users/kirstenziman/Documents/NEW_DIRECTORY/new_place/attention-memory-task/experiment1_blockCue/data'

for sub_dir in os.listdir(data_dir):
    sub_list.append(add_level(sum_pd(data_dir + '/' + sub_dir)))

In [4]:
exp1_raw = pd.concat(sub_list)
exp1_raw.to_csv('../parsed_data/behavioral_data_exp1.csv')

In [7]:
sub_list = []

data_dir = '/Users/kirstenziman/Documents/NEW_DIRECTORY/new_place/attention-memory-task/experiment2_trialCue/data'

for sub_dir in os.listdir(data_dir):
    sub_list.append(add_level(sum_pd(data_dir + '/' + sub_dir)))

In [8]:
exp2_raw = pd.concat(sub_list)
exp2_raw.to_csv('../parsed_data/behavioral_data_exp2.csv')